# This notebook outlines the final model tuning and set of predictions that ML Marauders have made for CS 181 Practical 1

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pickle
from sklearn.linear_model import Ridge, LogisticRegression, LogisticRegressionCV, Lasso
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import mean_squared_error

## Load and gently process the data (much of the preprocessing was done in FINAL.ipynb)

In [ ]:
# df_train = pd.read_csv('sam_data/rdk_feat_eng_whole_df_train_orig_features.csv')
# df_test = pd.read_csv('sam_data/rdk_feat_eng_whole_df_test_orig_features.csv')
# df_train = pd.read_csv('final_data/FINAL_train.csv')
# df_test = pd.read_csv('final_data/FINAL_test.csv')
df_train = pd.read_csv('FINAL_interactions/FINAL_train_25_interactions.csv')
df_test = pd.read_csv('FINAL_interactions/FINAL_test_25_interactions.csv')
df_train.head()

In [ ]:
df_test.head()

In [ ]:
# Drop the 'smiles' and 'Id' columns
df_train = df_train.drop(['smiles'], axis=1)
df_test = df_test.drop(['Id'], axis=1)
df_test = df_test.drop(['smiles'], axis=1)

# Store gap values
Y_train = df_train.gap.values

# Delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)
X_train = df_train.values
X_test = df_test.values
print "Train features:", X_train.shape, "Train gap:", Y_train.shape
print "Test features:", X_test.shape

In [ ]:
# Split training data into training and validation sets as well as begin some k-fold CV
cross_X_train, cross_X_valid, cross_Y_train, cross_Y_valid = train_test_split(X_train, Y_train, test_size=0.33, random_state=42)

#### For classification purposes, round target values to nearest .5

In [ ]:
# Round to nearest integer
# cross_Y_train_labels, cross_Y_valid_labels = np.round(cross_Y_train), np.round(cross_Y_valid)
# Y_train_labels = np.round(Y_train)
# Round to nearest .5
cross_Y_train_labels, cross_Y_valid_labels = (((np.round(2*cross_Y_train)/2.0)-0.5)/0.5).astype(int), (((np.round(2*cross_Y_valid)/2.0)-0.5)/0.5).astype(int)
Y_train_labels = (((np.round(2*Y_train)/2.0)-.5)/.5).astype(int)
# Round to nearest .25
# cross_Y_train_labels, cross_Y_valid_labels = (((np.round(4*cross_Y_train)/4.0)-.25)/.25).astype(int), (((np.round(4*cross_Y_valid)/4.0)-.25)/.25).astype(int)
# Y_train_labels = (((np.round(4*Y_train)/4.0)-.25)/.25).astype(int)

In [ ]:
print "'Training' features: ", cross_X_train.shape
print "'Validate' features: ", cross_X_valid.shape

# GOAL:

This notebook is set-up to chain together classification and regression methods. The thought is that we can, after we've trained the two models, to first apply a classifier to the data (in a clustering kind of sense) and then use the category or neighborhood that the sample is assigned as an additional feature to perform regression. Here the category or label will be the closest integer to the gap value. The idea behind this is to hijack the regression into a local region of the expected HOMO-LUMO gap based on the label. The hope is that this will pin the regressor closer to the right value. 

It's imperative that we get as accurate of a classifier as we can.

Fingers crossed.

### First: Let's build a classifier that will adequately label the samples

We'll start with Logistic Regression and try to fit the best model using a collection of C values

In [ ]:
%%time

logReg_training_acc = 0
logReg_test_acc = 0

Cs = [0.01, 0.1, 1.0]

for c in Cs:
    clf_logReg=LogisticRegression(penalty="l2",C=c, solver='lbfgs')
    clf_logReg.fit(cross_X_train,cross_Y_train_labels)
    training_acc = clf_logReg.score(cross_X_train,cross_Y_train_labels)
    test_acc = clf_logReg.score(cross_X_valid,cross_Y_valid_labels)
    print c, test_acc
    if logReg_test_acc < test_acc:
        logReg_test_acc = test_acc
        logReg_training_acc = training_acc
        best_logReg = clf_logReg

In [ ]:
logReg_training_acc = best_logReg.score(cross_X_train,Y_clf_train)
logReg_test_acc = best_logReg.score(cross_X_valid,Y_clf_valid)
print "Training Accuracy: %0.3f" % logReg_training_acc
print "Test Accuracy: %0.3f" % logReg_test_acc

#### Concatenate predicted labels onto test/validation set

In [ ]:
def adding_labels(feat_matrix,labels):
    '''Helper function that creates sparse binary array to concatenate to feat_matrix'''
    # Create empty matrix 
    added_cols = np.zeros((labels.shape[0],np.unique(labels).shape[0]))
    # Increment entry that corresponds to the sample having the specified label
    for ii in xrange(labels.shape[0]):
        added_cols[ii,labels[ii]] = 1
    # Concatenate label columns to feat_matrix
    feat_matrix = np.concatenate((feat_matrix,added_cols),axis=1)
    return feat_matrix

In [ ]:
cross_X_train = adding_labels(cross_X_train,cross_Y_train_labels)

cross_X_valid = adding_labels(cross_X_valid,cross_Y_valid_labels)

## Now generating a Random Forest Regression

In [ ]:
%%time
ExtraTrees_RMSE = 100
num_estimators = [10, 64, 96, 128]

    
for n_estimators in num_estimators:

    rf_reg = RandomForestRegressor(n_estimators=n_estimators, n_jobs=-1)
    rf_reg.fit(cross_X_train,cross_Y_train)
    y_pred = rf_reg.predict(cross_X_valid)

    RMSE = np.sqrt(mean_squared_error(cross_Y_valid, y_pred))
    if RMSE < ExtraTrees_RMSE:
        print n_estimators, RMSE
        RF_RMSE = RMSE
        RF_estimators = n_estimators
        best_RF = rf_reg
            
print "RandomForest with {0} estimators had RMSE of {1}".format(RF_estimators,RF_RMSE)        

## We're also going to tune a typical Linear Regression to have double coverage (either Ridge or Lasso)

In [ ]:
%%time
lasso_RMSE = 100
alphas = np.logspace(-4, 1, 30)

for alpha in alphas:
    lasso_reg = Lasso(alpha=alpha)
    lasso_reg.fit(cross_X_train, cross_Y_train)
    y_pred = lasso_reg.predict(cross_X_valid)
    
    RMSE = np.sqrt(mean_squared_error(cross_Y_valid,y_pred))
    if RMSE < lasso_RMSE:
        lasso_RMSE = RMSE
        lasso_alpha = alpha
        best_lasso = lasso_reg
        
print "Lasso RMSE: {0} with alpha: {1}".format(lasso_RMSE,lasso_alpha)

In [ ]:
# %%time
# ridge_RMSE = 100
# alphas = np.logspace(-4, 1, 30)

# for alpha in alphas:
#     ridge_clf = Ridge(alpha=alpha)
#     ridge_clf.fit(X_train_clf, cross_Y_train)
#     y_pred = ridge_clf.predict(X_valid_clf)
    
#     RMSE = np.sqrt(mean_squared_error(cross_Y_valid,y_pred))
#     if RMSE < ridge_RMSE:
#         ridge_RMSE = RMSE
#         ridge_alpha = alpha
#         best_ridge = ridge_clf
        
# print "Ridge RMSE: {0} with alpha: {1}".format(ridge_RMSE,ridge_alpha)

In [ ]:
with open('final2_classifier_and_regressors.pkl','w') as f:
    pickle.dump((best_logReg, best_RF, best_lasso),f)

In [ ]:
# with open('final_classifier_and_regressors.pkl','r') as fopen:
#     best_logReg, best_pcaExtraTrees, pcaExtraTrees_components, best_ridge = pickle.load(fopen)
    
# After opening this, you may need to re-configure the test and training set, that is if you have to restart the kernel

## Train on full training set, run on full test set

Here we will train the classifier and the regressions

In [ ]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

### Logistic Regression

In [ ]:
%%time
# Train classifier
best_logReg.fit(X_train,Y_train_labels)

# Run classifier on test set
label_pred = best_logReg.predict(X_test)

# Concatenate full training labels to full test set
X_train_clf = adding_labels(X_train,Y_train_labels)

# Concatenate predicted labels onto test set as a new feature
X_test_clf = adding_labels(X_test,label_pred)

print "Completed Classification of test set"

### Random Forest

In [ ]:
%%time
# Train ExtraTrees Regressor
best_RF.fit(X_train_clf,Y_train)

# Run ExtraTrees Regressor
RF_pred = best_RF.predict(X_test_clf)

# Save the ExtraTrees Predictions
write_to_file("extraTrees_FINAL2_TWK_10Feb.csv", RF_pred)

print "Completed Random Forest Regression"

### Lasso/Ridge Regression

In [ ]:
# # Train Ridge Regression
# best_ridge.fit(X_full_train_clf,Y_train)

# # Run Ridge Regressor
# ridge_pred = best_ridge.predict(X_test_clf)

# # Save the Ridge Predictions
# write_to_file("ridge_FINAL_TWK_10Feb.csv",ridge_pred)
# print "Completed Ridge Regression"

# Train Lasso Regression
best_lasso.fit(X_train_clf,Y_train)

# Run Lasso Regressor
lasso_pred = best_lasso.predict(X_test_clf)

# Save the Lasso Predictions
write_to_file("lasso_FINAL2_TWK_10Feb.csv",lasso_pred)
print "Completed Lasso Regression"